In [1]:
#%pip install --upgrade langchain langsmith langgraph langchain_openai openai python-dotenv javascript numpy
# run this in terminal: python3 -m javascript --install @near-lake/primitives
%reload_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os
from langchain_core.messages import HumanMessage
from graph.trivial_agent_graph import IndexerAgentGraphBuilder
from agents.IndexerAgent import indexer_agent_model, tool_executor
from tools.JavaScriptRunner import tool_js_on_block_schema_func, tool_js_on_block_schema
from agents.BlockExtractorAgent import run_js_on_block_only_schema
import json

# Load .env file
load_dotenv('.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Indexer Agent Local"


# Execute Data Extractor Agent

In [69]:
from tools.NearLake import tool_get_block_heights
from agents.BlockExtractorAgent import block_extractor_agent_model

inputs = {
    "messages": [
        HumanMessage(
            content="Extract all FunctionCalls where receiverId is 'app.nearcrowd.near'. For each function call, I need signerId, block height, receiptId, block datetime, methodName. Also add all fields from args that are decoded from base64-encoded JSON. I only need actions for successful receipts.",
        )
    ]
}

# Define the tools that the agent will use
tools = [tool_js_on_block_schema_func, tool_get_block_heights]
model = block_extractor_agent_model(tools)

builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

js_code_res = app.with_config({"run_name": "TrivialGraph block_extractor_agent_model"}).invoke(inputs)

In [43]:
from langchain_utils import get_tool_call_arguments
from agents.BlockExtractorAgent import JsResponse

js_response = get_tool_call_arguments(js_code_res['messages'], JsResponse)

js_response.js_schema

"{'type': 'array'}"

In [44]:
from agents.DDLAgent import ddl_generator_agent_model

model = ddl_generator_agent_model()

inputs = {
    "messages": [
        HumanMessage(
            content="Here is the schema from JS developer: " + js_response.js_schema,
        )
    ]
}
builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

ddl_code_res = app.with_config({"run_name": "Graph ddl_generator_agent_model", "recursion_limit": 2}).invoke(inputs)


GraphRecursionError: Recursion limit of 2 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

In [26]:
ddl_code_res

{'messages': [HumanMessage(content="Here is the schema from JS developer: {'type': 'array', 'items': {'type': 'object', 'properties': {'signerId': {'type': 'string'}, 'blockHeight': {'type': 'integer'}, 'receiptId': {'type': 'string'}, 'blockDatetime': {'type': 'string'}, 'methodName': {'type': 'string'}, 'task_ordinal': {'type': 'integer'}, 'task_hash': {'type': 'array', 'items': {'type': 'integer'}}}}}"),
  AIMessage(content='Based on the provided schema, the DDL script for creating a table in a Postgres database would be:\n\n```sql\nCREATE TABLE blocks (\n    signer_id VARCHAR(255),\n    block_height INTEGER,\n    receipt_id VARCHAR(255),\n    block_datetime TIMESTAMP,\n    method_name VARCHAR(255),\n    task_ordinal INTEGER,\n    task_hash INTEGER[]\n);\n```\n\nThis script creates a table named `blocks` with the following columns:\n\n- `signer_id` of type `VARCHAR(255)`\n- `block_height` of type `INTEGER`\n- `receipt_id` of type `VARCHAR(255)`\n- `block_datetime` of type `TIMESTAMP

In [27]:
from agents.DDLAgent import DDLAgentResponse

ddl_response = get_tool_call_arguments(ddl_code_res['messages'], DDLAgentResponse)
print(ddl_response)

KeyError: 'tool_calls'

# Scratchpad
Cells below are for testing and debugging

In [68]:
from tools.JavaScriptRunner import fetch_block, get_function_calls_from_block
from tools.contract_block_heights import get_block_heights

block_heights = get_block_heights("pool.near", 5)

block_heights

get_function_calls_from_block(block_heights[0], 'pool.near')


'[{"args": "{\\"staking_pool_id\\":\\"nearone.pool.near\\",\\"owner_id\\":\\"\\",\\"stake_public_key\\":\\"ed25519:HYRRAMtyXPCVHdyk89WsRyrirbUKXe7CBmLfFzZGSwtm\\",\\"reward_fee_fraction\\":{\\"numerator\\":10,\\"denominator\\":100}}", "deposit": "4000000000000000000000000", "gas": 300000000000000, "methodName": "create_staking_pool"}]'

In [41]:

from tools.JavaScriptRunner import run_js_on_block_only_schema, run_js_on_block
code = """
function extractData(block) {
    const actions = block.actions();
    const receipts = block.receipts();
    const header = block.header();

    const successfulReceipts = receipts.filter(receipt => receipt.status.SuccessValue);
    const filteredActions = actions.filter(action => action.receiverId === 'app.nearcrowd.near' && action.operations.some(op => op.FunctionCall));

    const result = [];

    for (const action of filteredActions) {
      for (const operation of action.operations) {
        if (operation.FunctionCall) {
          const receipt = receipts.find(receipt => receipt.receiptId === action.receiptId);
          if (receipt) {
            const args = JSON.parse(atob(operation.FunctionCall.args));
            result.push({
              signerId: action.signerId,
              blockHeight: header.height,
              receiptId: action.receiptId,
              receipt: receipt,
              blockDatetime: new Date(parseInt(header.timestampNanosec) / 1000000),
              methodName: operation.FunctionCall.methodName,
              ...args
            });
          }
        }
      }
    }

    return result;
  }
  return extractData(block);
"""
print(json.dumps(run_js_on_block(119688212, code)))

"('evaluateWithContext', 'SyntaxError: Invalid or unexpected token\\n    at Object.evaluateWithContext (/Users/pavel-near/sources/near/indexer-agent/.venv/lib/python3.12/site-packages/javascript/js/bridge.js:70:29)\\n    at Bridge.call (/Users/pavel-near/sources/near/indexer-agent/.venv/lib/python3.12/site-packages/javascript/js/bridge.js:136:42)\\n    at Bridge.onMessage (/Users/pavel-near/sources/near/indexer-agent/.venv/lib/python3.12/site-packages/javascript/js/bridge.js:231:25)\\n    at Socket.<anonymous> (/Users/pavel-near/sources/near/indexer-agent/.venv/lib/python3.12/site-packages/javascript/js/bridge.js:292:18)\\n    at Socket.emit (node:events:514:28)\\n    at addChunk (node:internal/streams/readable:324:12)\\n    at readableAddChunk (node:internal/streams/readable:297:9)\\n    at Readable.push (node:internal/streams/readable:234:10)\\n    at Pipe.onStreamRead (node:internal/stream_base_commons:190:23)')"
